### Goal is to look into the large scale tokenization of the entire dataset


#### Takeaways: We CAN TF-IDF 2 million, but memory is a big issue? That on top of not playing around with embeddings yet is a slightly sussy thought.

In [1]:
import pandas as pd
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from datasets import load_dataset
import datasets
import torch
from collections import Counter
import string
from scipy import spatial
import nltk
from nltk.stem import RegexpStemmer

from nltk.corpus import stopwords

c:\Users\halom\anaconda3\envs\test2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_dict = datasets.load_from_disk("./Dataset/CodeSearchCorpus/")
train_dataset = dataset_dict["train"]

In [3]:
# train_dataset.to_pandas() Not feasible, 90% memory

In [4]:
# train_dataset

In [5]:
num_rows = 100000
np.random.seed(1)
train_subset_indices = np.random.choice(len(train_dataset), num_rows, replace = False)
train_dataset_subset = train_dataset.select(train_subset_indices)
len(train_dataset_subset)


100000

In [6]:
train_dataset_subset["func_name"][:100]

['load_config',
 'DotFormatter.appendDot',
 'ControlHighestGeneratedTickImpl.getTraceSummaryLine',
 'CanDeleteDirectory',
 'Solve.pinv',
 'RegistriesInner.listUsagesAsync',
 'Number.formatSize',
 'PropertyReaderAccessor.hasPropertyValue',
 '',
 'BeanSpec.beanSpecOf',
 'csvdir_bundle',
 'Verify',
 'buildQuery',
 'StatsServlet.doPost',
 'Install',
 'GetBlockByHash',
 'FirstAndLastSequence',
 'get_coredump_network_config',
 'Serve',
 'VCard.addName',
 'LaravelHtmlEncryptServiceProvider.boot',
 'SyncClient.download',
 'resolve_frompath',
 'DB2Statement.bind',
 'AbstractParametrizedValidator.validateParameterCount',
 'ArticleStatisticImporter.importArticleStatistics',
 'PEROBS.BigArray.delete_if',
 'Phrase.from',
 'SetAcceptLanguage',
 'CosmosClient.__QueryFeed',
 'Herald.send',
 'ezIBpy.cancelHistoricalData',
 'Widget.registerAssets',
 '',
 'WSSubject.setInvocationSubject',
 '',
 'capture_payment',
 'status_codes_by_date_stats',
 'WOrientConf.createFromApplicationConf',
 'parsePropertyFunc

In [7]:
train_dataset_subset = train_dataset_subset.select_columns("func_documentation_string")

In [8]:
# train_dataset_subset = train_dataset_subset.select_columns("func_documentation_string")

In [9]:
# train_dataset_subset.to_pandas()

In [10]:
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

st = RegexpStemmer('ing$|s$|e$|able$', min=4)
tokenizer = RegexpTokenizer(r'\w+')
# tokenizer.tokenize('Eighty-seven miles to go, yet.  Onward!')

In [11]:
train_dataset_subset = train_dataset_subset.to_pandas()

In [12]:
# nltk.download('stopwords')

In [13]:
stop_words = set(stopwords.words('english'))

In [14]:
train_dataset_subset["func_doc_tokens"] = train_dataset_subset["func_documentation_string"].apply(lambda x: tokenizer.tokenize(x))

train_dataset_subset["func_doc_stem_tokens"] = train_dataset_subset["func_doc_tokens"].apply(lambda x: [st.stem(word.lower()) for word in x if word not in stop_words])
# train_dataset_subset["func_doc_stem_tokens"] = train_dataset_subset["func_doc_tokens"].apply(lambda x: [st.stem(word) for word in x if word not in stop_words])
# ~35 seconds

In [15]:
train_dataset_subset["func_doc_stem_tokens"][0]

['Load',
 'yaml',
 'configuration',
 'param',
 'path',
 'pathlib',
 'Path',
 'object',
 'point',
 'configuration']

In [16]:
# [st.stem(word) for word in train_dataset_subset["func_doc_tokens"][0] if word not in stop_words]

In [17]:
# [st.stem(word) for word in train_dataset_subset["func_doc_tokens"][0]]

In [18]:
train_dataset_subset["func_doc_stem_tokens"]

0        [Load, yaml, configuration, param, path, pathl...
1        [Convert, AST, dot, representation, append, gi...
2        [Get, summary, trac, lin, messag, Javadoc, des...
3        [CanDeleteDirectory, return, nil, TODO, check,...
4        [Compute, pseudo, invers, Not, function, use, ...
                               ...                        
99995    [Load, object, JSON, fil, saved, py, func, Con...
99996    [LogRequest, create, request, logger, goa, mid...
99997    [Construct, new, acces, control, policy, condi...
99998    [Return, dict, contain, subset, namespac, pref...
99999    [restAddVirtualHost, parse, payload, add, vhos...
Name: func_doc_stem_tokens, Length: 100000, dtype: object

In [19]:
from nltk.util import ngrams

In [20]:
train_dataset_subset["func_doc_stem_tokens"][0]
list(ngrams(train_dataset_subset["func_doc_stem_tokens"][0], 2))

[('Load', 'yaml'),
 ('yaml', 'configuration'),
 ('configuration', 'param'),
 ('param', 'path'),
 ('path', 'pathlib'),
 ('pathlib', 'Path'),
 ('Path', 'object'),
 ('object', 'point'),
 ('point', 'configuration')]

In [21]:
bigram_lst = []
for i in range(100000):
    bigram_lst += list(ngrams(train_dataset_subset.iloc[i]["func_doc_stem_tokens"], 2))

In [22]:
bigram_counter = Counter(bigram_lst)


In [23]:
len(bigram_counter)
test = bigram_counter.values()

In [24]:
import statistics

In [25]:
list(test)

[2,
 3,
 197,
 270,
 2,
 6,
 13,
 9,
 1,
 1,
 1,
 4,
 2,
 9,
 19,
 94,
 58,
 1,
 1,
 1,
 1,
 2,
 81,
 1283,
 1,
 1,
 1,
 1,
 2,
 22,
 1,
 3,
 1,
 1,
 4,
 3,
 5,
 4,
 6,
 14,
 14,
 3,
 3,
 1,
 96,
 1,
 4,
 12,
 2,
 2,
 2,
 30,
 66,
 2,
 2,
 3,
 4,
 2,
 2,
 3,
 3,
 2,
 2,
 69,
 2,
 2,
 33,
 15,
 4,
 7,
 6,
 12,
 8,
 4,
 1,
 1,
 16,
 48,
 41,
 257,
 204,
 1703,
 350,
 500,
 24,
 18,
 109,
 13,
 13,
 36,
 93,
 93,
 93,
 93,
 93,
 96,
 93,
 90,
 804,
 443,
 432,
 430,
 430,
 325,
 467,
 93,
 95,
 1,
 1,
 1,
 2,
 2,
 4,
 2885,
 108,
 30,
 17,
 37,
 71,
 1,
 2,
 77,
 14325,
 1,
 1,
 1,
 2,
 1,
 1,
 4,
 1,
 2,
 7,
 1,
 2,
 1,
 19,
 3957,
 22,
 6,
 13,
 1,
 2,
 385,
 21,
 1,
 2,
 134,
 1,
 2,
 1,
 1,
 1,
 1,
 3,
 2,
 1,
 9,
 1,
 98,
 74,
 5,
 3,
 1,
 61,
 69,
 20,
 1,
 1,
 11,
 7,
 2,
 3,
 3,
 7,
 1,
 2,
 3,
 49,
 1,
 1,
 1,
 8,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1664,
 1,
 1,
 1771,
 159,
 51,
 12,
 2,
 2,
 2,
 1,
 3,
 8,
 205,
 9,
 1,
 1,
 9,
 1,
 1,
 1,


In [26]:
# bigram_counter = (Counter({k: c for k, c in bigram_counter.items() if c <= 1000 and c >= 50}))
# bigram_counter = bigram_counter
# len(bigram_counter)
# bigram_counter[('valu', 'param')]
bigram_counter
len(bigram_counter)
bigram_set = set(list(bigram_counter.keys()))


In [27]:
bigram_set

{('Propertie', 'bind'),
 ('Check', 'snapshot'),
 ('param', 'strOperator'),
 ('window', 'arg'),
 ('lin', 'allowed'),
 ('CSDL', 'hash'),
 ('Add', 'fuzzy'),
 ('fill', 'CreateBasicAuthSecretOption'),
 ('function', 'trie'),
 ('Environment', 'VariableContext'),
 ('registry_api', 'get'),
 ('SimpleWorkflowBehavior', 'param'),
 ('present', 'ZK'),
 ('non', 'set'),
 ('notify', 'fals'),
 ('empty', 'rout'),
 ('tupl', 'SourceLocation'),
 ('URL', 'zum'),
 ('wsinc5', 'Output'),
 ('param', 'dd'),
 ('All', 'AND'),
 ('earlier', 'param'),
 ('sourc', 'constraint'),
 ('23bar', 'foo'),
 ('limit', 'cut'),
 ('Output', 'PASSED'),
 ('attribute', 'script'),
 ('entity', 'valu'),
 ('internal', 'hidden'),
 ('one', 'ths'),
 ('Operation', 'getCorporationsCorporationIdStarbasesAsync'),
 ('function', 'network'),
 ('Observatory', 'file'),
 ('incomplet', 'Ther'),
 ('upload', 'artifact'),
 ('lik', 'Alternatively'),
 ('tool', 'id'),
 ('Any', 'implement'),
 ('hub', 'subscription'),
 ('claus', 'current'),
 ('td', 'AT'),
 ('09

In [28]:
len(bigram_set), ('load', 'yaml') in bigram_set

(903923, True)

In [29]:
inverted_index = {}
for i in range(len(train_dataset_subset)):
    token_counter = Counter(train_dataset_subset.iloc[i]["func_doc_stem_tokens"])
    bigram_counter_i = Counter(list(ngrams(train_dataset_subset.iloc[i]["func_doc_stem_tokens"], 2)))
    total_len = sum(token_counter.values()) + sum(bigram_counter_i.values())
    for token in token_counter:
        if token not in inverted_index:
            inverted_index[token] = {}
        inverted_index[token][i] = (token_counter[token], total_len)
    
    for bigram in bigram_counter_i:
        if bigram in bigram_set:
            if bigram not in inverted_index:
                inverted_index[bigram] = {}
            inverted_index[bigram][i] = (bigram_counter_i[bigram],total_len)

In [30]:
inverted_index["yaml"]
# inverted_index["load"]

{0: (1, 19),
 1929: (4, 247),
 2100: (3, 303),
 3190: (1, 47),
 3907: (1, 103),
 7993: (2, 85),
 9183: (1, 193),
 10113: (1, 129),
 10315: (1, 35),
 12164: (1, 49),
 12972: (1, 37),
 13108: (1, 21),
 14314: (1, 17),
 15135: (2, 211),
 15618: (1, 145),
 16318: (1, 229),
 16853: (2, 93),
 17075: (3, 175),
 17432: (1, 119),
 17682: (1, 11),
 18073: (1, 213),
 19180: (1, 155),
 20987: (11, 121),
 24885: (2, 299),
 24949: (1, 101),
 26596: (1, 91),
 26718: (2, 97),
 27494: (1, 125),
 29347: (2, 29),
 29581: (1, 47),
 29645: (2, 61),
 30786: (1, 21),
 31020: (1, 115),
 31313: (1, 61),
 31839: (1, 293),
 32559: (1, 1095),
 33400: (1, 41),
 33880: (1, 55),
 34148: (1, 241),
 34223: (1, 25),
 34791: (3, 337),
 34991: (1, 43),
 35281: (1, 71),
 35626: (1, 31),
 37330: (1, 133),
 38067: (1, 217),
 38592: (1, 121),
 39686: (1, 75),
 39993: (1, 31),
 40065: (1, 21),
 43722: (1, 17),
 43950: (1, 147),
 45385: (6, 159),
 45392: (1, 139),
 48872: (1, 29),
 51146: (5, 123),
 51233: (1, 11),
 51868: (1,

In [ ]:
# inverted_index
# for bigram in bigram_counter:
#     print(bigram)

# inverted_index[('to', 'draw')]

tf_idf = {}
for i in range(len(train_dataset_subset)):
    # print(i)
    tokens = train_dataset_subset["func_doc_stem_tokens"].iloc[i]
    
    bigram_lst_i = []
    for bigram in list(ngrams(train_dataset_subset.iloc[i]["func_doc_stem_tokens"], 2)):
        if bigram in bigram_set:
            bigram_lst_i.append(bigram)
    tokens += bigram_lst_i
    # print(tokens)
    counter = Counter(tokens)
    words_count = len(tokens)

    for token in set(tokens):
        tf = counter[token] / words_count
        df = len(inverted_index[token])
        idf = np.log((len(train_dataset_subset) + 1) / (df + 1))

        tf_idf[i, token] = tf * idf


In [31]:
def query_results_NEW(query_string, inverted_index, k = 10):
    query_tokens = [st.stem(word) for word in tokenizer.tokenize(query_string) if word not in stop_words]
    # print(query_tokens)
    rel_indices = []
    
    answer_dict = {} 
    for token in query_tokens:
        if token in inverted_index:
            rel_indices = list(set(inverted_index[token].keys()))

            for rel_i in rel_indices:
                if rel_i not in answer_dict:
                    answer_dict[rel_i] = 0
                # print(inverted_index[token], token)
                tf = inverted_index[token][rel_i][0] / inverted_index[token][rel_i][1]
                df = len(inverted_index[token])
                idf = np.log((len(train_dataset_subset) + 1) / (df + 1))
                answer_dict[rel_i] += tf * idf

    
    bigram_lst = list(ngrams(query_tokens, 2))
    for bigram in bigram_lst:
        if bigram in inverted_index:
            rel_indices = list(set(inverted_index[bigram].keys()))

            for rel_i in rel_indices:
                if rel_i not in answer_dict:
                    answer_dict[rel_i] = 0
                
                    tf = inverted_index[bigram][rel_i][0] / inverted_index[bigram][rel_i][1]
                    df = len(inverted_index[bigram])
                    idf = np.log((len(train_dataset_subset) + 1) / (df + 1))
                    answer_dict[rel_i] += 2 * tf * idf
                
                    # print(bigram, rel_i)

    # len(query_embedding[0])
    # len(tsed_DF["embeddings"][0])
    
    # print(query_tokens)
    # print(len(rel_indices))

    # print(answer_dict)
    result_lst = [(a,b) for (a,b) in answer_dict.items()]
    result_lst.sort(reverse=True, key = lambda x: x[1])
    return result_lst[:k]

    

In [32]:
def query_results(query_string, inverted_index, tf_idf, k = 10):
    query_tokens = [st.stem(word) for word in tokenizer.tokenize(query_string) if word not in stop_words]
    # print(query_tokens)
    rel_indices = []
    
    # answer_dict = 
    for token in query_tokens:
        if token in inverted_index:
            rel_indices = list(inverted_index[token].keys())
    
    bigram_lst = list(ngrams(query_tokens, 2))
    for bigram in bigram_lst:
        if bigram in inverted_index:
            # print(bigram)
            # print(list(inverted_index[bigram].keys()))
            query_tokens.append(bigram)
            rel_indices += list(inverted_index[bigram].keys())
    
    rel_indices = set(rel_indices)

    # len(query_embedding[0])
    # len(tsed_DF["embeddings"][0])
    
    print(query_tokens)
    print(len(rel_indices))
    result_lst = []
    for i in rel_indices:
        tf_score = 0
        for token in query_tokens:
            if (i, token) in tf_idf:
                if type(token) != str:
                    print("HELPPPP", i, tf_idf[(i, token)])
                    tf_score += 2 * (tf_idf[(i, token)])
                else:
                    # print("No")
                    tf_score += (tf_idf[(i, token)])
            # try:
            #     # print(tf_idf[(i, token)])
            #     tf_score += (tf_idf[(i, token)])
            # except: 
            #     # print(token)
            #     continue #this is bad, make sure this isn't the play
        # print(i)

        

        result_lst.append([i, tf_score])
    
    result_lst.sort(reverse=True, key = lambda x: x[1])
    return result_lst[:k]

In [33]:
# tf_idf[3760]

In [34]:
query_results_NEW("string to datetime", inverted_index, k = 10)

# [[39418, 2.4423054773840445],
#  [11453, 2.4423054773840445],
#  [44109, 2.0352545644867033],
#  [98562, 1.7445039124171742],
#  [55271, 1.7445039124171742],
#  [8798, 1.5264409233650276],
#  [87076, 1.5264409233650276],
#  [19040, 1.4366502808141435],
#  [94558, 1.4366502808141435],
#  [86413, 1.4090223907984871]]

[(39418, 1.3568363763244689),
 (11453, 1.3568363763244689),
 (44109, 1.2211527386920222),
 (98562, 0.9393482605323247),
 (55271, 0.9393482605323247),
 (87076, 0.8722519562085871),
 (8798, 0.8722519562085871),
 (86413, 0.8696721338044758),
 (94558, 0.7400925689042558),
 (19040, 0.7290464111594162)]

In [ ]:
train_dataset_subset.iloc[11453]["func_documentation_string"]

In [ ]:
inverted_index[('str', 'datetim')]

In [ ]:
train_dataset_subset.iloc[3760]["func_documentation_string"]

In [ ]:
"save list to file"
query_results("save list to file", inverted_index, tf_idf, k = 10)

In [ ]:
train_dataset_subset.iloc[97660]["func_documentation_string"]

In [ ]:
# tf_idf
# len(inverted_index)#144089

In [ ]:
# 144089 - 138465

In [ ]:
# tf_idf

In [ ]:
# train_dataset = train_dataset.select_columns("func_documentation_string")

# train_dataset

In [ ]:
# train_dataset = train_dataset.map(lambda tkn: {"func_doc_tokens": tokenizer.tokenize(tkn)})

In [ ]:
# train_dataset_DF = train_dataset.to_pandas()

In [ ]:
# tokenizer.tokenize(test_sentence)

In [ ]:
train_dataset_DF["func_doc_tokens"] = train_dataset_DF["func_documentation_string"].apply(lambda x: tokenizer.tokenize(x))

#25.3 seconds

In [ ]:
train_dataset_DF["func_doc_stem_tokens"] = train_dataset_DF["func_doc_tokens"].apply(lambda x: [st.stem(word) for word in x])
# ~35 seconds

In [ ]:
# test_tokens
# [ps.stem(word) for word in test_tokens]

In [ ]:
inverted_index = {}
for i in range(len(train_dataset_DF)):
    token_counter = Counter(train_dataset_DF.iloc[i]["func_doc_stem_tokens"])

    for token in token_counter:
        if token not in inverted_index:
            inverted_index[token] = {}
        inverted_index[token][i] = token_counter[token]

#Took ~2 minutes

In [ ]:
# len(inverted_index)

In [ ]:
#73622 No stemming
#68995 with stemming
#Pickle afterwards
# with open("./pickleObjects/inverted_index_FULL.pkl", 'wb') as f:  # open a text file
#     pickle.dump(inverted_index, f) # serialize the list
#     f.close()

In [ ]:
# with open("./pickleObjects/tfidf_FULL.pkl", 'wb') as f:  # open a text file
#     pickle.dump(tf_idf, f) # serialize the list
#     f.close()

In [ ]:
with open('./pickleObjects/inverted_index_FULL.pkl', 'rb') as f:
    inverted_index = pickle.load(f) # deserialize using load()
    f.close()

In [ ]:
tf_idf = {}
for i in range(len(train_dataset_DF)):
    # print(i)
    tokens = train_dataset_DF["func_doc_stem_tokens"].iloc[i]
    counter = Counter(tokens)
    words_count = len(tokens)

    for token in np.unique(tokens):
        tf = counter[token] / words_count
        df = len(inverted_index[token])
        idf = np.log((len(train_dataset_DF) + 1) / (df + 1))

        tf_idf[i, token] = tf * idf

# 4 minutes there or thereabouts

In [ ]:
# with open("./pickleObjects/tfidf_FULL.pkl", 'wb') as f:  # open a text file
#     pickle.dump(tf_idf, f) # serialize the list
#     f.close()

In [ ]:
# with open('./pickleObjects/tfidf_FULL.pkl', 'rb') as f:
#     tf_idf = pickle.load(f) # deserialize using load()
#     f.close()

In [ ]:
def query_results(query_string, inverted_index, tf_idf, k = 10):
    query_tokens = [st.stem(word) for word in tokenizer.tokenize(query_string)]
    print(query_tokens)
    rel_indices = []
    
    for token in query_tokens:
        if token in inverted_index:
            rel_indices += list(inverted_index[token].keys())
    
    rel_indices = set(rel_indices)

    # len(query_embedding[0])
    # len(tsed_DF["embeddings"][0])
    

    result_lst = []
    for i in rel_indices:
        for token in query_tokens:
            tf_score = 0

            # if (i, token) in tf_idf:
            #     tf_score += (tf_idf[(i, token)])
            try:
                # print(tf_idf[(i, token)])
                tf_score += (tf_idf[(i, token)])
            except: 
                continue #this is bad, make sure this isn't the play
        # print(i)

        result_lst.append([i, tf_score])
    
    result_lst.sort(reverse=True, key = lambda x: x[1])
    return result_lst[:k]

In [ ]:
query_results("string to datetime", inverted_index, tf_idf, k = 50)

In [ ]:
train_dataset[100930]

In [ ]:
print((0, 'str') in tf_idf)

In [ ]:
(0, 'Chosen') in tf_idf
# 'Chosen' in tf_idf